### pandas test

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('orders.csv')
df['date'] = df['date'].astype('category')
df['spec'] = df['spec'].astype('category')
df.head(10)

,date,vendor_id,chain_id,city_id,spec,successful_orders,fail_orders
0,2019-07-02,40065,7501,23,Суши,54.0,1.0
1,2019-08-27,48058,33226,23,Шаурма,0.0,0.0
2,2019-09-25,35430,26220,25,Пицца,2.0,0.0
3,2019-09-21,56553,38601,23,Шашлыки,1.0,0.0
4,2019-09-21,43919,30984,25,Суши,6.0,0.0
5,2019-06-13,53006,36214,26,Грузинская,2.0,0.0
6,2019-07-07,28982,20651,24,Суши,0.0,1.0
7,2019-08-01,45472,30409,24,Бургеры,37.0,4.0
8,2019-07-03,65358,43034,23,Русская,1.0,0.0
9,2019-09-19,60878,41231,23,Русская,4.0,0.0


Сколько в датафрейме колонок с типом **float64**?

`2 columns with Dtype == float64`

Чему равно произведение кол-ва наблюдений на кол-во колонок в датафрейме?

`7 columns * 96118 entries = 672826`

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96118 entries, 0 to 96117
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   date               96118 non-null  category
 1   vendor_id          96118 non-null  int64   
 2   chain_id           96118 non-null  int64   
 3   city_id            96118 non-null  int64   
 4   spec               95733 non-null  category
 5   successful_orders  96118 non-null  float64 
 6   fail_orders        96118 non-null  float64 
dtypes: category(2), float64(2), int64(3)
memory usage: 3.9 MB


Сколько уникальных городов в датафрейме?

`4 different cities`

In [4]:
df['city_id'].unique().shape[0]

4

Сколько успешных заказов в среднем получали рестораны в день? (ответ округлить до целого)

`~9 successful orders per day`

In [5]:
df['successful_orders'].mean()

8.877400694979087

Кол-во ресторанов со специализацией "Рыба"?

`18 restaurants`

In [6]:
df[df['spec'] == 'Рыба']['vendor_id'].unique().shape[0]

18

Сколько у ресторана 40065 было дней с кол-вом успешных заказов менее 20?

`vendor 40065 has 45 days with less than 20 successful orders`

In [7]:
vendor_40065 = df[df['vendor_id'] == 40065]
vendor_40065[vendor_40065['successful_orders'] < 20].shape[0]

45

Сколько успешных заказов в среднем получили рестораны за весь период времени? (ответ округлить до целого)

`555`

In [8]:
df.groupby(by='vendor_id')['successful_orders'].sum().mean()

555.1581001951854

Город с самым маленьким количеством успешных заказов за весь период.

`city 24`

In [9]:
df.groupby(by='city_id')['successful_orders'].sum()

city_id
23    277744.0
24    114977.0
25    228518.0
26    232039.0
Name: successful_orders, dtype: float64

Какая специализация имеет самую высокую долю неуспешных заказов за весь период времени (все заказы по конкретному ресторану складываются из успешных, неуспешных заказов)?

`Вегетарианское меню -- the lowest success_prob -> the highest fail_prob`

In [10]:
spec = df.groupby(by='spec')
success_prob = spec['successful_orders'].sum() / (spec['successful_orders'].sum() + spec['fail_orders'].sum())
success_prob.sort_values().head()

spec
Вегетарианское меню    0.172414
Халяль                 0.782609
Хинкали                0.833333
Завтраки               0.844523
Хот-доги               0.859259
dtype: float64

Какая из сетей (chain_id) показала максимальный абсолютный рост за август 2019 года по сравнению с июлем 2019 года? Учитывать только те сети, у которых заказы были в июле и в августе?

```python
july_mask = (df.date >= '2019-07-01') & (df.date <= '2019-07-31')
august_mask = (df.date >='2019-08-01') & (df.date <= '2019-08-31')
july_sales = df[july_mask].groupby('chain_id')['successful_orders'].sum().reset_index()
august_sales = df[august_mask].groupby('chain_id')['successful_orders'].sum().reset_index()
merged_table = august_sales.merge(july_sales, how='left', on='chain_id', suffixes=('_august', '_july'))
merged_table['increase'] = merged_table['successful_orders_august'] - merged_table['successful_orders_july']
merged_table.sort_values('increase', ascending = False).iloc[0]['chain_id']
```

`29925`